# Creating of a Flask application showing Citibike stations


In this segment we will create a basic app, where we will connect to the Citibike database, and display the list of stations. 

Let's remember first how to get data from the database.

In [1]:
# This code creates a connection to the database
import MySQLdb as mdb

con = mdb.connect(host = 'localhost', 
                  user = 'root',
                  database = 'citibike_new',
                  passwd = 'dwdstudent2015', 
                  charset='utf8', use_unicode=True);

In [4]:
cur = con.cursor(mdb.cursors.DictCursor)
cur.execute("SELECT station_id, name, capacity, lon, lat FROM Stations LIMIT 5")
stations = cur.fetchall()
cur.close()

for s in stations:
    print(s)
    print("-----")

{'lon': -73.9939, 'lat': 40.7673, 'capacity': 39, 'name': 'W 52 St & 11 Ave', 'station_id': 72}
-----
{'lon': -74.0067, 'lat': 40.7191, 'capacity': 33, 'name': 'Franklin St & W Broadway', 'station_id': 79}
-----
{'lon': -74.0002, 'lat': 40.7112, 'capacity': 27, 'name': 'St James Pl & Pearl St', 'station_id': 82}
-----
{'lon': -73.9763, 'lat': 40.6838, 'capacity': 62, 'name': 'Atlantic Ave & Fort Greene Pl', 'station_id': 83}
-----
{'lon': -74.0015, 'lat': 40.7418, 'capacity': 39, 'name': 'W 17 St & 8 Ave', 'station_id': 116}
-----


## Displaying the database results in HTML

Now, we are going to display these data in HTML page in a table format. An HTML table is defined with the [`<table>` tag](https://www.w3schools.com/html/html_tables.asp). Each table row is defined with the `<tr>` tag. A table header is defined with the `<th>` tag. By default, table headings are bold and centered. A table data/cell is defined with the `<td>` tag. 


Let's transfer the code for records retrieving to the `"webserver.py"` file:

```python
from flask import Flask, render_template
import MySQLdb as mdb

app = Flask(__name__)
    
@app.route('/citibike')
def citibike_stations():
    
    con = mdb.connect(host = 'localhost', 
                  user = 'root',
                  database = 'citibike_new',
                  passwd = 'dwdstudent2015', 
                  charset='utf8', use_unicode=True);
    
    cur = con.cursor(mdb.cursors.DictCursor)
    cur.execute("SELECT station_id, name, capacity, lon, lat FROM Stations")
    stations = cur.fetchall()
    cur.close()
    con.close()

    return render_template('citibike.html', stations=stations)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)
```



and add the following code to the `"templates/citibike.html"` file

```html
<!DOCTYPE html>
<html>
<body>   <!-- This is a comment in HTML -->
    <table>
        <thead>
            <tr>    <!-- Construct table header -->
                <th>Station ID</th>
                <th>Address</th>
                <th>Capacity</th>
                <th>Lon / Lat</th>
            </tr>
        </thead>
        <tbody>
            {% for station in stations %}    <!-- Fill table rows for each station --> 
                <tr>
                    <td>{{ station.station_id }}</td>
                    <td>{{ station.name }}</td>
                    <td>{{ station.capacity }}</td>
                    <td>{{ station.lon }}, {{ station.lat }}</td>
           
                </tr>
            {% endfor %}
         </tbody>
    </table>
</body>
</html>
```

Run this code to see how the table looks.

In [6]:
%run webserver.py

ERROR:root:File `'webserver.py'` not found.


## Getting Parameters

Now let's see how we can query for the status of a Citibike station over time.

For this part, we want to create a new function, where we will pass the `station_id` as a **parameter**. Then our code will read the value of the parameter `station_id` and then will query the database to get the status of that station.

The related pieces of code for reading a parameter are the following

> `from flask import request`

> `station_id = request.args.get('station_id')`

Now, let's add the import command `from flask import request` into the `webserver.py` and also add the function below that adds an extra URL `station_status`. 

```python
@app.route('/station_status')
def station_status():
    
    station_id = int(request.args.get('station_id'))
    
    con = mdb.connect(host = 'localhost', 
                  user = 'root',
                  database = 'citibike_new',
                  passwd = 'dwdstudent2015', 
                  charset='utf8', use_unicode=True);
    
    cur = con.cursor(mdb.cursors.DictCursor)
    query = '''SELECT num_bikes_available, 
                      num_docks_available, 
                      num_bikes_disabled, 
                      num_docks_disabled, 
                      last_reported 
               FROM Status
               WHERE station_id = %s'''
    cur.execute(query, (station_id,) )
    status = cur.fetchall()
    cur.close()
    con.close()

    return render_template('station_status.html', station_id = station_id, statuses=status)
```

And we can create the corresponding template file under `templates/station_status.html` for showing the results:
    
```html
<!DOCTYPE html>
<html>
<body> 
    <table>
        <thead>
            <tr>    <!-- Construct table header -->
                    <th> Time </th>
                    <th> Bikes available </th>
                    <th> Docks available </th>
                    <th> Bikes disabled </th>
                    <th> Docks disabled </th>
            </tr>
        </thead>
        <tbody>
            {% for status in statuses %}
                <tr>
                    <td>{{ status.last_reported }}</td>
                    <td>{{ status.num_bikes_available }}</td>
                    <td>{{ status.num_docks_available }}</td>
                    <td>{{ status.num_bikes_disabled }}</td>
                    <td>{{ status.num_docks_disabled }}</td>
                </tr>
            {% endfor %}
         </tbody>
    </table>
</body>
</html>
```

This function creates a new URL, to which we can pass a parameter `station_id` as follows:

> http://your_ip_address:5000/station_status?station_id=72



In [10]:
%run webserver.py

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
INFO:werkzeug: * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
 * Restarting with stat
INFO:werkzeug: * Restarting with stat
